In [3]:
import pandas as pd


In [4]:
df = pd.read_csv('../src/data/user_aggregates.csv')


In [5]:
df.head()

,MSISDN/Number,Number of xDR Sessions,Total Session Duration (ms),Total Download (Bytes),Total Upload (Bytes),Total Data Volume (Bytes),YouTube Download (Bytes),YouTube Upload (Bytes),Netflix Download (Bytes),Netflix Upload (Bytes),Gaming Download (Bytes),Gaming Upload (Bytes),Other Download (Bytes),Other Upload (Bytes)
0,3.360100e+10,1,116720.0,8.426375e+08,36053108.0,8.786906e+08,14900201.0,6724347.0,10265105.0,16915876.0,8.110911e+08,1367528.0,377096990.0,9473882.0
1,3.360100e+10,1,181230.0,1.207552e+08,36104459.0,1.568596e+08,5324251.0,7107972.0,770569.0,10451194.0,1.050353e+08,14714780.0,279557701.0,2152370.0
2,3.360100e+10,1,134969.0,5.566597e+08,39306820.0,5.959665e+08,2137272.0,19196298.0,16525919.0,2827981.0,5.290685e+08,9759228.0,495086501.0,6607171.0
3,3.360101e+10,1,49878.0,4.019932e+08,20327526.0,4.223207e+08,4393123.0,2584198.0,1157362.0,784730.0,3.880748e+08,3051292.0,25248001.0,10031701.0
4,3.360101e+10,2,37104.0,1.363130e+09,94280527.0,1.457411e+09,10339971.0,31193031.0,24971647.0,24230077.0,1.308981e+09,5816727.0,777643713.0,27160771.0


In [7]:
print(df.info())  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106856 entries, 0 to 106855
Data columns (total 14 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   MSISDN/Number                106856 non-null  float64
 1   Number of xDR Sessions       106856 non-null  int64  
 2   Total Session Duration (ms)  106856 non-null  float64
 3   Total Download (Bytes)       106856 non-null  float64
 4   Total Upload (Bytes)         106856 non-null  float64
 5   Total Data Volume (Bytes)    106856 non-null  float64
 6   YouTube Download (Bytes)     106856 non-null  float64
 7   YouTube Upload (Bytes)       106856 non-null  float64
 8   Netflix Download (Bytes)     106856 non-null  float64
 9   Netflix Upload (Bytes)       106856 non-null  float64
 10  Gaming Download (Bytes)      106856 non-null  float64
 11  Gaming Upload (Bytes)        106856 non-null  float64
 12  Other Download (Bytes)       106856 non-null  float64
 13 

In [8]:
print(df.describe())

       MSISDN/Number  Number of xDR Sessions  Total Session Duration (ms)  \
count   1.068560e+05           106856.000000                 1.068560e+05   
mean    4.511474e+10                1.389777                 1.461672e+05   
std     2.889423e+12                0.809863                 1.863587e+05   
min     3.360100e+10                0.000000                 7.142000e+03   
25%     3.365088e+10                1.000000                 7.130800e+04   
50%     3.366365e+10                1.000000                 1.027400e+05   
75%     3.368344e+10                2.000000                 1.727990e+05   
max     8.823971e+14               18.000000                 1.855375e+07   

       Total Download (Bytes)  Total Upload (Bytes)  \
count            1.068560e+05          1.068560e+05   
mean             6.336527e+08          5.730942e+07   
std              4.645551e+08          3.565765e+07   
min              8.827082e+06          2.866892e+06   
25%              3.148271e+08  